# Efficient Quantum Expectation Values Estimation for Concentrated States

## Summary

Measuring  [expectation values](https://en.wikipedia.org/wiki/Expectation_value_(quantum_mechanics)) of [observables](https://en.wikipedia.org/wiki/Observable#Quantum_mechanics) is essential in Variational Quantum Algorithms  [[1]](https://www.nature.com/articles/s42254-021-00348-9) (VQAs). These hybrid quantum-classical algorithms are the best hope for obtaining quantum advantage in the current era. However, many applications require such high precision in estimating the expectation values of observables that makes VQAs impractical. 

In this project, we deal we the efficient measurement of the expectation values of observables acting on concentrate states, i.e., when only a small number of amplitudes in a measurement basis are non-negligible. This is accomplished using a hybrid quantum-classical algorithm based on a novel procedure described in [[2]](https://en.wikipedia.org/wiki/Variational_quantum_eigensolver) that includes improvements like reducing the number of CNOT gates employed and the estimated number of parameters. This algorithm is supported in Qiskit, and its operation is detailed in this tutorial, providing the theoretical framework and numerical simulations to evaluate its performance.


# 1. Introduction

The current generation of quantum processors is characterized by limited numbers of qubits, low gate fidelities, short coherence times, and noise processes that limit circuit depth. In this context, the VQAs is the best hope for obtaining quantum advantage. Those algorithms typically utilize quantum computers to measure expectation values of observables for a given state realized by a parametrized [quantum circuit](https://en.wikipedia.org/wiki/Quantum_circuit). However, precisely estimating these expectation values requires numerous [measurement](https://en.wikipedia.org/wiki/Measurement_in_quantum_mechanics) repetitions to reduce statistical fluctuation. Many applications require such high precision that VQA algorithms are impractical due to the enormous amount of time needed. For instance, quantum chemical calculations require the so-called chemical accuracy of $ 1 \textrm{ kcal/mol } \approx 1.6 \times 10^{-3}$ Hartree, making the [Variational Quantum Eigensolver](https://en.wikipedia.org/wiki/Variational_quantum_eigensolver) (VQE) method non-viable even for a small number of molecules [[3]](https://journals.aps.org/prresearch/abstract/10.1103/PhysRevResearch.4.033154).

In this project, we implement a hybrid quantum-classical algorithm supported on Quiskit to measure the expectation values of observables efficiently. This algorithm is designed to be effective when the state is concentrated, which is a family of quantum states with interesting practical applications, such as in Quantum Chemistry. The approach is based on the paper [Quantum expectation-value estimation by computational basis sampling](https://link.aps.org/doi/10.1103/PhysRevResearch.4.033173) and includes improvements such as a reduction in the number of CNOT gates used and the estimated number of parameters. To explain the algorithm, this tutorial briefly describes the computation of the expectation values of observables. The algorithm's functioning is then described, and its implementation in Quiskit is shown using instructive examples. Moreover, we assess its performance by calculating the resources required to achieve a fixed accuracy. Finally, we illustrate how to implement the VQE method for a molecular Hamiltonian.

## 2.  Measuring expectation values of observables

Let us consider a $n$-qubit system described by the pure state
\begin{align}
|\psi \rangle = \sum_{i=0}^{d-1} c_i |i \rangle,
\end{align}

where $i = i_{n-1} ... i_1 i_0$ is the integer representation of a binary string and $d = 2^n$ is the dimension of the system. The coefficients $c_i$ are complex numbers such that $\sum_{i=0}^{d-1} |c_i|^2 =1$. An observable $O$ acting on $d$ qubits can be expressed as a linear combination of products of Pauli and identity operators on single qubits by

\begin{align}
 O = \sum_{k=1}^M  \alpha_k  P_k,
\end{align}

where $P_k \in \{I, X,Y, Z\}^{\otimes d}$, $\alpha_k\in\mathbb{R}$, and $M$ is the number of the Pauli strings. Therefore, the expectation value of $O$ can be written as

\begin{align}
\langle \psi  | O | \psi \rangle
= \sum_{k=1}^M  \alpha_k \langle \psi  | P_k | \psi \rangle.
\end{align}

In the simplest way to estimate the expectation value $\langle \psi  | O |\psi \rangle$, each
of $\alpha_k \langle \psi  | P_k | \psi \rangle$ is measured on a quantum computer and then
assembled by classical postprocessing. However, for many applications, the number of quantities $M$ to be measured is $\mathcal{O}\left(\mathrm{poly}(d)\right)$, making the computation unfeasible if the system dimension $d$ is not small enough.
There are several studies to reduce the number of measurements to estimate expectation values of observables. Those works, to the best of our knowledge, are driven by: grouping Pauli strings [[4]](https://dx.doi.org/10.1088/1367-2630/18/2/023023), using classical shadows [[5]](https://www.nature.com/articles/s41567-020-0932-7#citeas), and dealing with one part of the quantum state classically [[6]](https://arxiv.org/abs/2106.04755). Despite these efforts, the measurement counts for practical applications are not significantly reduced.

## 3.  Efficient quantum expectation values estimation for concentrated states

### 3.1 Motivation
Under the assumption that $|\psi \rangle$ is a concentrated state, it is defined a reduced state $|\psi_R \rangle$ as

\begin{align}
|\psi_R \rangle = \mathcal{N}_R \sum_{r=1}^{R} c_{i_r} |i_r \rangle,
\end{align}

where $R$ is a fixed number, $|c_{i_1}|^2 \geq |c_{i_2}|^2 \dots \geq |c_{i_R}|^2 \geq \max\{|c_{i}|^2: i \neq i_r \text{ for each } r=1\dots R\}$, and $\mathcal{N}_r = 1/\sqrt{ \sum_{r=1}^{R} |c_{i_r}|^2}$. Therefore, the expectation value of an observable $O$ acting on the system can be approximated by

\begin{equation}
\begin{split}
\langle \psi  | O | \psi \rangle \approx \mathcal{N}_R^2 \langle \psi_R  | O | \psi_R \rangle
= & \,\mathcal{N}_R^2 \sum_{r,r'=1}^R  c_{i_r} c_{i_r'}^* \, \langle i_{r'}  | O | i_r \rangle \\
= & \, \mathcal{N}_R^2 \left\{ \sum_{r=1}^R |c_{i_r}|^2 \langle i_r  | O | i_r \rangle + 2 \sum_{r < r'} \mathrm{Re}(c_{i_r} c_{i_r'}^*) \, \mathrm{Re} \langle i_{r'}  | O | i_r \rangle  -  2 \sum_{ r < r'} \mathrm{Im}(c_{i_r} c_{i_r'}^*) \, \mathrm{Im} \langle i_{r'}  | O | i_r \rangle \right\},
\end{split}
\end{equation}

Here the cuantities $\langle i_{r'} |O | i_r \rangle$ can be calculated classically whenever the observable is a tensor product of one-qubit observable. Hence, by an estimation of the coefficients $|c_{i_r}|^2$ , $2\mathrm{Re}(c_{i_r} c_{i_r'}^*)$ and $2\mathrm{Im}(c_{i_r} c_{i_r'}^*)$ we can estimate the expectation value of $O$. For this purpose, we define a POVM $E$ composed of the elements
\begin{equation}
\begin{split}
P_k = K |k \rangle \langle k |, \quad 
D_{jk} = \frac{K}{2}(|j \rangle  + |k \rangle ) ( \langle  j| + \langle k |), & \quad
A_{jk} =  \frac{K}{2}(|j \rangle  - |k \rangle ) ( \langle  j| - \langle k |), \quad
L_{jk} = \frac{K}{2}(|j \rangle  + i|k \rangle ) ( \langle  j| -i \langle k |), \\
R_{jk} = & \frac{K}{2}(|j \rangle  - i|k \rangle ) ( \langle  j| + i \langle k |), 
\end{split}
\end{equation}

and noticing that
 
\begin{align}
|c_{i}|^2 =   \, \frac{\mathrm{Tr}\left(\rho P_{j} \right)}{K},\qquad
2 \mathrm{Re}(c_j c_k^*) =  \, \frac{\mathrm{Tr}\left(\rho D_{jk} \right) - \mathrm{Tr}\left(\rho A_{jk} \right)}{K},\qquad
-2 \mathrm{Im}(c_j c_k^*) =  \, \frac{\mathrm{Tr}\left(\rho L_{jk} \right) - \mathrm{Tr}\left(\rho R_{jk} \right)}{K},
\end{align}

we approximate $\langle \psi  | O | \psi \rangle$ by measuring the POVM $E$. 

$\textbf{Observation:}$ Since concentrate states usually present interactions for a small number of bodies,  there are some quantities $\langle i_{r'}  | O | i_r \rangle = 0 $. Therefore, some coefficients are negligible due to the structure of the quantum state. This reduction of coefficients depend on the number of $X$ and $Y$ gates on the Pauli string, and improves the required measurement resources to compute the approximation, hence improving the estimation of $\langle \psi  | O | \psi \rangle$.


### 3.2 The method

To implement the above idea we reproduce the following procedure:

1. Prepare the state $|\psi\rangle$ on quantum computers, and measure it in the computational basis $\{| i \rangle : i = 0,1,2^n-1\}$ with a number $L_f$ of shoots.

2. From the outcomes of the measurement we  pick up the most frequent R elements such that $1-|\langle \psi_R | \psi \rangle|^2$ be lower of some previusly defined tolerance $\mathrm{Tol}$ ($\approx 10^{-4}$). Next, we sort in descending order of frecuency the selected elements.

3. Determine the number of $X$ and $Y$ gates in the Pauli string to reduce the number of coefficients in the estimation.

4. Measure The POVM $E$ and compute the non-negligible coefficientes $|c_{i_r}|^2$ , $2\,\mathrm{Re}(c_{i_r} c_{i_r'}^*)$ and $2\,\mathrm{Im}(c_{i_r} c_{i_r'}^*)$ as a sample mean of the outcomes.

The described method inherits the performance properties shown in [[2]](https://link.aps.org/doi/10.1103/PhysRevResearch.4.033173). First, the error in estimating  $\langle \psi  | O | \psi \rangle$ produced by  approximating  the quantum estate by $| \psi_R\rangle$ is proportional to $\sqrt{1-|\langle \psi_R | \psi \rangle|^2}$. Next, the number of measurements necessary to attain some precision in the standard deviation of the approximation is fewer than that of the alternative techniques. Moreover, the described method improves that reported in [[2]](https://link.aps.org/doi/10.1103/PhysRevResearch.4.033173), since the number of CNOT gates employed is not $\mathcal{O}(d)$, but is equal to the number of $X$ and $Y$ gate in the Pauli string.

## 4. Implementation using Qiskit


## 5. VQE method for molecular hamiltonians 

## 7. Discussion and outlook

We implement a hybrid quantum-classical algorithm supported by Quiskit to measure the expectation values of observables efficiently. This Algorithm is effective when the state is concentrated, a very interesting family of quantum states in practical application, for instance, Quantum Chemistry. The approach presents some improvements to that presented in [[2]](https://link.aps.org/doi/10.1103/PhysRevResearch.4.033173) like a reduction in the number of CNOT gates used and a reduction in the estimated number of parameters. The implementation of the algorithm in Quiskit is detailed in this tutorial, where several numerical simulations evaluate its performance.

There are some improvements that could be made on the algorithm implementation:
 
 - Implementing the algorithm in Quiskit is not yet fully polished, as some subroutines are not optimally implemented, causing the program to run slower and consume more resources. 
 
 - The estimator's variance has a closed form, calculated by the program. However, we were not able to perform numerical simulations
 
 - The optimal allocation of the measurement budget was not used because it did not affect the estimation outcome.  We can deduce from this that some minor error caused the program to malfunction.

 - The VQE implementation does not support higher molecules, since the ansatz...



## References 

[1] M. Cerezo, Andrew Arrasmith, Ryan Babbush, Simon C. Benjamin, Suguru Endo, Keisuke Fujii, Jarrod R. McClean, Kosuke Mitarai, Xiao Yuan, Lukasz Cincio & Patrick J. Coles. Variational quantum algorithms. ***Nature Reviews Physics***. 3:625–644, 8 2021. ISSN 2522-5820. [doi: 10.1038/s42254-021-00348-9](https://www.nature.com/articles/s42254-021-00348-9).

[2] Kohda, Masaya and Imai, Ryosuke and Kanno, Keita and Mitarai, Kosuke and Mizukami, Wataru and Nakagawa, Yuya O. Quantum expectation-value estimation by computational basis sampling. ***Phys. Rev. Res***, 4:033173, Sep 2022. doi:
10.1103/PhysRevResearch.4.033173. URL https://link.aps.org/doi/10.1103/PhysRevResearch.4.033173.

[3] Gonthier, J\'er\^ome F. and Radin, Maxwell D. and Buda, Corneliu and Doskocil, Eric J. and Abuan, Clena M. and Romero, Jhonathan. Measurements as a roadblock to near-term prac-tical quantum advantage in chemistry: Resource analysis. ***Phys. Rev. Res.***, 4:033154, Aug 2022. doi: 10.1103/PhysRevResearch.4.033154.  URL https://link.aps.org/doi/10.1103/PhysRevResearch.4.033154

[4] Jarrod R McClean, Jonathan Romero, Ryan Babbush, and Al ́an Aspuru-Guzik. The theory of variational hybrid
quantum-classical algorithms. ***New Journal of Physics***, 18(2):023023, feb 2016. doi: 10.1088/1367-2630/18/2/
023023. URL https://dx.doi.org/10.1088/1367-2630/18/2/023023

[5] sin-Yuan Huang, Richard Kueng, and John Preskill. Predicting many properties of a quantum system from very
few measurements. ***Nature Physics***, 16:1050–1057, 10 2020. ISSN 1745-2473. [doi: 10.1038/s41567-020-0932-7](https://www.nature.com/articles/s41567-020-0932-7#citeas).

[6] M. D. Radin and P. Johnson, Classically-boosted variational quantum eigensolver, [arXiv:2106.04755](https://arxiv.org/abs/2106.04755).
